# 데이터 불러오기

In [1]:
import pandas as pd
import os

pd.options.display.float_format = '{:.3f}'.format
pd.set_option('display.max_columns', None)

PATH = './data/'

file_list= os.listdir(PATH)
csv_list = list()

for file in file_list:
    if file.split(".")[-1] == 'csv':
        csv_list.append(file)
data = ['living_popul', 'resident_popul', 'area', 'store', 'working_popul', 'facilitie', 'estimated_sales']

for name, file in zip(data, csv_list):
    globals()[name] = pd.read_csv(PATH + file, encoding = "ansi")

store = store.drop("Unnamed: 0", axis = 1)
estimated_sales = estimated_sales.drop("Unnamed: 0", axis = 1)

# 위/경도 데이터 확인

In [2]:
area.head()

,상권_구분_코드_명,상권_코드_명,엑스좌표_값,와이좌표_값,자치구_코드_명,행정동_코드_명,영역_면적
0,골목상권,배화여자대학교(박노수미술관),197093,453418,종로구,청운효자동,149264
1,골목상권,자하문터널,196991,455057,종로구,부암동,178306
2,골목상권,평창동서측,197064,456643,종로구,평창동,369415
3,골목상권,정독도서관,198581,453781,종로구,가회동,83855
4,골목상권,중앙고등학교,198883,453690,종로구,가회동,166872


In [3]:
area.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1650 entries, 0 to 1649
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   상권_구분_코드_명  1650 non-null   object
 1   상권_코드_명     1650 non-null   object
 2   엑스좌표_값      1650 non-null   int64 
 3   와이좌표_값      1650 non-null   int64 
 4   자치구_코드_명    1650 non-null   object
 5   행정동_코드_명    1650 non-null   object
 6   영역_면적       1650 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 90.4+ KB


# 지도

In [4]:
import pandas as pd
import folium

In [5]:
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Proj, transform

geometry = [Point(lon, lat) for lon, lat in zip(area['엑스좌표_값'], area['와이좌표_값'])]

geo_area = gpd.GeoDataFrame(area, geometry=geometry, crs=' EPSG:5181')

# UTM-K 좌표계에서 WGS 84 좌표계로 변환
wgs84 = Proj(init='epsg:4326')
utm_k = Proj(init='epsg:5181')

geo_area['엑스좌표_값'], geo_area['와이좌표_값'] = transform(utm_k, wgs84, geo_area['geometry'].x.values, geo_area['geometry'].y.values)

geo_area

C:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\Admin\AppData\Local\Temp\ipykernel_8352\4084700737.py:13: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotc

,상권_구분_코드_명,상권_코드_명,엑스좌표_값,와이좌표_값,자치구_코드_명,행정동_코드_명,영역_면적,geometry
0,골목상권,배화여자대학교(박노수미술관),126.967,37.580,종로구,청운효자동,149264,POINT (197093.000 453418.000)
1,골목상권,자하문터널,126.966,37.595,종로구,부암동,178306,POINT (196991.000 455057.000)
2,골목상권,평창동서측,126.967,37.609,종로구,평창동,369415,POINT (197064.000 456643.000)
3,골목상권,정독도서관,126.984,37.584,종로구,가회동,83855,POINT (198581.000 453781.000)
4,골목상권,중앙고등학교,126.987,37.583,종로구,가회동,166872,POINT (198883.000 453690.000)
...,...,...,...,...,...,...,...,...
1645,관광특구,명동 남대문 북창동 다동 무교동 관광특구,126.982,37.564,중구,소공동,983618,POINT (198397.000 451614.000)
1646,관광특구,동대문패션타운 관광특구,127.011,37.567,중구,광희동,606058,POINT (200996.000 451976.000)
1647,관광특구,종로·청계 관광특구,126.998,37.570,종로구,종로1·2·3·4가동,653127,POINT (199796.000 452274.000)
1648,관광특구,잠실 관광특구,127.115,37.516,송파구,방이2동,2462734,POINT (210188.000 446334.000)


In [6]:
from folium import Marker

map_info = geo_area[["엑스좌표_값", "와이좌표_값", "상권_코드_명"]].dropna()
map_info

map = folium.Map(
    location=[37.552289, 126.991792],
    zoom_start=12
)

for lat, long, name in zip(map_info["와이좌표_값"], map_info["엑스좌표_값"], map_info["상권_코드_명"]):
    folium.Marker([lat, long], popup=name, tooltip=name).add_to(map)

map

In [7]:
area.groupby('자치구_코드_명').size().reset_index(name='Count').sort_values('Count', ascending= False)

,자치구_코드_명,Count
0,강남구,103
19,영등포구,96
10,동대문구,77
12,마포구,77
16,성북구,76
4,관악구,74
22,종로구,74
14,서초구,74
3,강서구,70
17,송파구,70
